In [ ]:
!pip install benepar
!pip install cython numpy
!pip install Keras
!pip install wordninja


     |████████████████████████████████| 3.3MB 31.6MB/s 
     |████████████████████████████████| 2.1MB 37.6MB/s 
     |████████████████████████████████| 1.2MB 42.8MB/s 
     |████████████████████████████████| 901kB 39.3MB/s 
  Created wheel for benepar: filename=benepar-0.2.0-cp37-none-any.whl size=37647 sha256=53aafaa935f5f6b89425f2672bd61927a29137de23e04fc8019f4f3882942b00
  Stored in directory: /root/.cache/pip/wheels/dd/90/28/513063023646df7774be9401c440a5f13b48bdbab15a67fc42
Successfully built benepar
     |████████████████████████████████| 542kB 18.5MB/s 
  Created wheel for wordninja: filename=wordninja-2.0.0-cp37-none-any.whl size=541554 sha256=f102d4a7cd87ad51820124bc5000d8e39eb51c4a1d4df8b0d0342b537733bcaa
  Stored in directory: /root/.cache/pip/wheels/22/46/06/9b6d10ed02c85e93c3bb33ac50e2d368b2586248f192a2e22a
Successfully built wordninja


In [ ]:
import pandas as pd 
import os
import nltk
from nltk.tokenize import word_tokenize 
from nltk import Tree
from nltk.draw.tree import TreeView
import os
from spacy import displacy
import spacy
from collections import Counter
import en_core_web_sm
from keras.preprocessing.text import text_to_word_sequence
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
import wordninja

nltk.download('wordnet')
nlp = en_core_web_sm.load()
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
path_data = '/content/drive/My Drive/'
path = os.path.join(path_data, 'stock_data.csv')
datastock = pd.read_csv(path,encoding='latin-1',names=['News', 'Sentiment'], header=0)
datastock.Sentiment[datastock.Sentiment == 1] = 'positive'
datastock.Sentiment[datastock.Sentiment == -1] = 'negative'


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
path_data = '/content/drive/My Drive/Colab Notebooks/financial news'
path = os.path.join(path_data, 'all-data.csv')
datakaggle = pd.read_csv(path,encoding='latin-1',names=['Sentiment', 'News'])

In [ ]:
data= pd.concat([datastock, datakaggle], ignore_index = True)
data.reset_index()

,index,News,Sentiment
0,0,Kickers on my watchlist XIDE TIT SOQ PNK CPW B...,positive
1,1,user: AAP MOVIE. 55% return for the FEA/GEED i...,positive
2,2,user I'd be afraid to short AMZN - they are lo...,positive
3,3,MNTA Over 12.00,positive
4,4,OI Over 21.37,positive
...,...,...,...
10632,10632,LONDON MarketWatch -- Share prices ended lower...,negative
10633,10633,Rinkuskiai 's beer sales fell by 6.5 per cent ...,neutral
10634,10634,Operating profit fell to EUR 35.4 mn from EUR ...,negative
10635,10635,Net sales of the Paper segment decreased to EU...,negative


In [ ]:
# !pip install english-words
# from english_words import english_words_lower_alpha_set#english_words_set
# def splitConcatenatedWords(word):
#   new
#   if len(token[0])>1 and len(wordninja.split(token[0]))>1:
#     for splitword in wordninja.split(token[0]):
#       if splitword in english_words_set is True:
        
#       else:
          

Sentence Object


In [ ]:
class Sentence:
  def __init__(self):
    self.words = []
    self.wholeSentence = None
    self.sentiment = None
    self.tree = None
    self.NerTag = None

  def appendWords(self, word): 
    self.words.append(word) 

class Word:
  def __init__(self):
    self.word = None
    self.lemma = None
    self.synsetName = None
    self.synsetDefinition = None
    self.posTag = None
    self.grammarTag = None
    self.synonyma = []
    self.antonyma = None
    self.negation = None


In [ ]:


def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

def get_first_sense(word, pos=None):
    if pos:
        synsets = wordnet.synsets(word,pos)
    else:
        synsets = wordnet.synsets(word)
    if not synsets:
      return None
    else:
      return synsets[0]

data['Postag-Lemmatized Sentence'] = None
data['Pos-Lem Sen without tags'] = None
listofSentences = []
for index,row in data.iterrows():
  #inicializácia objektov
  lm = WordNetLemmatizer()
  sentenceObject = Sentence()

  newEditedSentence = ""
  taggedlematizedsentence = []


  sentence = row['News']
  sentenceObject.sentiment = row['Sentiment']
  tokenized_sentence = text_to_word_sequence(sentence)
  tagged_sentence = nltk.pos_tag(tokenized_sentence)

  for token in tagged_sentence:
    wordObject = Word()
    best_synset = None
    postag = get_wordnet_pos(token[1])

    wordObject.word = token[0]
    if postag is None or postag is '': # not supply tag in case of None
      lemma = lm.lemmatize(token[0])
      taggedlematizedsentence.append((lemma,token[1]))
      newEditedSentence+=lemma + " "
      best_synset = get_first_sense(lemma)
      wordObject.lemma = lemma
      wordObject.posTag = token[1]
    else:
      lemma = lm.lemmatize(token[0], postag) 
      taggedlematizedsentence.append((lemma,token[1])) # vráti sa mi lemma a pridam povodny postag nie ten ktory mam z wordnetu 
      best_synset = get_first_sense(lemma,postag)

      newEditedSentence+=lemma + " "
      wordObject.lemma = lemma
      wordObject.posTag = token[1]

    if best_synset is None:
      #print("Slovo nema viac významov") 
      wordObject.synsetName = None
      wordObject.synsetDefinition = None
      #TODO negation handling   
    else:
      wordObject.synsetName = best_synset.name()
      wordObject.synsetDefinition = best_synset.definition()
      for synlemma in best_synset.lemmas():

        if synlemma.name() == lemma:
          continue
        else:
          wordObject.synonyma.append(synlemma.name())
        if synlemma.antonyms():
          wordObject.antonyma = synlemma.antonyms()[0].name()
          #print(wordObject.lemma,synlemma.antonyms()[0].name())


    sentenceObject.appendWords(wordObject)
  sentenceObject.wholeSentence = newEditedSentence
  data.at[index,'Postag-Lemmatized Sentence'] = taggedlematizedsentence
  data.at[index,'Pos-Lem Sen without tags'] = newEditedSentence
  data.at[index,'Sentence-word object'] = sentenceObject
  listofSentences.append(sentenceObject)

In [ ]:
count = 0
for sentence in listofSentences:
  for word in sentence.words:
    count+=1
print(len(listofSentences))
print(len(data))

10637
10637


## ***Constituency parsing(nie dependency)***


In [ ]:
data['Sentence-Regex-Tree'] = None

chunkGram = r"""
                       NP: {<DT>?<JJ>*<NN>}    #To extract Noun Phrases
                       P: {<IN>}               #To extract Prepositions
                       V: {<V.*>}              #To extract Verbs
                       PP: {<P> <NP>}          #To extract Prepostional Phrases
                       VP: {<V> <NP|PP>*}      #To extarct Verb Phrases
                       """
for index,row in data.iterrows():
  cp = nltk.RegexpParser(chunkGram)
  result = cp.parse(row['Postag-Lemmatized Sentence'])
  tree = Tree.fromstring(str(result))
  data.at[index,'Sentence-Regex-Tree'] = tree

  if index == 1: 
      tree.pretty_print()

                                                                                                                                         S                                                                                                                                                   
        _________________________________________________________________________________________________________________________________|_______________________________________________________________________________________________________________________________________________     
       |           |       |       |       |      |       |      |        |         |       |        |            |          |           |             |        |          |            |      |           |              |               VP                          |                  |   
       |           |       |       |       |      |       |      |        |         |       |        |            |          |           |   

*Dependency* parsing

In [ ]:

sp = spacy.load('en_core_web_sm')
for index,row in data.iterrows():
  sentence = sp(row['News'])
  # print(sentence)
  if index < 1 :
    displacy.render(sentence, style='dep',jupyter=True,  options={'distance': 85}) # displacy.render  & jupyter=True, -pridat, ked chcem aby to zobrazilo v notebooku a nie na na novej html stranke

NER taggs


In [ ]:
print(data['News'][1])
daco = dict([(str(x), x.label_) for x in nlp(str(data['News'][1])).ents])
print(daco)

Technopolis plans to develop in stages an area of no less than 100,000 square meters in order to host companies working in computer technologies and telecommunications , the statement said .
{'Technopolis': 'GPE', '100,000 square meters': 'QUANTITY'}


/usr/local/lib/python3.7/dist-packages/torch/distributions/distribution.py:46: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  'with `validate_args=False` to turn off validation.')


Experimental constituent-based analysis and dependency parsing 


In [ ]:
%tensorflow_version 1.x
!pip install benepar
import benepar
benepar.download('benepar_en3')
from benepar.spacy_plugin import BeneparComponent
import benepar, spacy

# Loading spaCy’s en model and adding benepar model to its pipeline
nlp = spacy.load('en')
if spacy.__version__.startswith('2'):
        nlp.add_pipe(benepar.BeneparComponent("benepar_en3"))
else:
        nlp.add_pipe("benepar", config={"model": "benepar_en3"})
data['Sentence-Benepar-Tree'] = None

for index,row in data.iterrows():
  # Generovanie stromu pre jednotliv=e texty 
  doc = nlp(row['Pos-Lem Sen without tags'])
  sent = list(doc.sents)[0]

  # print(sent._.parse_string)
  bp = sent._.parse_string
  benepartree = Tree.fromstring(str(bp))
  data.at[index,'Sentence-Benepar-Tree'] = benepartree

  if index == 1: 

    benepartree.pretty_print()
    

TensorFlow is already loaded. Please restart the runtime to change versions.
[nltk_data] Downloading package benepar_en3 to /root/nltk_data...
[nltk_data]   Package benepar_en3 is already up-to-date!


/usr/local/lib/python3.7/dist-packages/torch/distributions/distribution.py:46: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  'with `validate_args=False` to turn off validation.')


                  S                                                                                                                                                                
      ____________|___________                                                                                                                                                      
     |                        VP                                                                                                                                                   
     |        ________________|__________________________                                                                                                                           
     |       |                                           S                                                                                                                         
     |       |                                           |                                        

In [ ]:
for index,sentence in enumerate(listofSentences):
  for word in sentence.words:
    print(word.lemma)

In [ ]:
import pickle 
filehandler = open("/content/drive/My Drive/Sentence", 'wb') 
pickle.dump(listofSentences, filehandler, pickle.HIGHEST_PROTOCOL)
filehandler.close()

In [ ]:
len(data)

10637

In [ ]:
data.to_csv('data_preprocessed.csv')
!cp data_preprocessed.csv "/content/drive/My Drive/"
